# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Откройте и изучите файл

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


## 2. Разбейте данные на выборки

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
df_test, df_valid = train_test_split(df_valid, test_size=0.5, random_state=12345)
df_train.shape

(1928, 5)

In [ ]:
df_valid.shape

(643, 5)

In [ ]:
df_test.shape

(643, 5)

## 3. Исследуйте модели

In [ ]:
# создадим переменные для признаков и целевого признака
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid =df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra'] 
features_test =df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [ ]:
#Исследуем модели дерева решений
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
#Исследуем глубину дерева решений
for max_depth in range(1,11):
  model = DecisionTreeClassifier(random_state=12345, max_depth=max_depth)
  model.fit(features_train, target_train)
  model_predict = model.predict(features_valid)
  accuracy = accuracy_score(target_valid, model_predict)
  print('max_depth=', max_depth,':', accuracy)

max_depth= 1 : 0.7356143079315708
max_depth= 2 : 0.7744945567651633
max_depth= 3 : 0.7791601866251944
max_depth= 4 : 0.7744945567651633
max_depth= 5 : 0.7838258164852255
max_depth= 6 : 0.776049766718507
max_depth= 7 : 0.7993779160186625
max_depth= 8 : 0.7931570762052877
max_depth= 9 : 0.7807153965785381
max_depth= 10 : 0.7884914463452566


**Выбираем max_depth=7**

In [ ]:
#Исследуем влияние параметров min_samples_split и min_samples_leaf
for split in [2,10,20,30,40]:
  for leaf in [2,8,14,16,17,20]:
    model = DecisionTreeClassifier(criterion='entropy', 
                                    random_state=12345, max_depth=6,min_samples_split=split, min_samples_leaf=leaf)     
    model.fit(features_train, target_train)
    modelPredictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, modelPredictions)
    print('split=',split,'leaf=',leaf,':', accuracy)

split= 2 leaf= 2 : 0.7807153965785381
split= 2 leaf= 8 : 0.7838258164852255
split= 2 leaf= 14 : 0.7822706065318819
split= 2 leaf= 16 : 0.7916018662519441
split= 2 leaf= 17 : 0.7916018662519441
split= 2 leaf= 20 : 0.7916018662519441
split= 10 leaf= 2 : 0.7791601866251944
split= 10 leaf= 8 : 0.7838258164852255
split= 10 leaf= 14 : 0.7822706065318819
split= 10 leaf= 16 : 0.7916018662519441
split= 10 leaf= 17 : 0.7916018662519441
split= 10 leaf= 20 : 0.7916018662519441
split= 20 leaf= 2 : 0.7853810264385692
split= 20 leaf= 8 : 0.7838258164852255
split= 20 leaf= 14 : 0.7822706065318819
split= 20 leaf= 16 : 0.7916018662519441
split= 20 leaf= 17 : 0.7916018662519441
split= 20 leaf= 20 : 0.7916018662519441
split= 30 leaf= 2 : 0.7853810264385692
split= 30 leaf= 8 : 0.7838258164852255
split= 30 leaf= 14 : 0.7822706065318819
split= 30 leaf= 16 : 0.7916018662519441
split= 30 leaf= 17 : 0.7916018662519441
split= 30 leaf= 20 : 0.7916018662519441
split= 40 leaf= 2 : 0.7884914463452566
split= 40 leaf=

**Выбираем параметр min_samples_split=2(по умолчанию) и min_samples_leaf=16**

In [ ]:
model = DecisionTreeClassifier(random_state=12345, max_depth=7, min_samples_leaf=16)
model.fit(features_train, target_train)
model_pred = model.predict(features_valid)
accuracy = accuracy_score(target_valid, model_pred)
print('Точность дерева решений вылидационной выборки:', accuracy)

Точность дерева решений вылидационной выборки: 0.7978227060653188


In [ ]:
#Исследуем модели случайного леса
from sklearn.ensemble import RandomForestClassifier
for estim in [1,4,12,20,40,80,90]:
  model = RandomForestClassifier(random_state=12345, n_estimators=estim)
  model.fit(features_train, target_train) 
  model_pred = model.predict(features_valid)
  accuracy = accuracy_score(target_valid, model_pred)
  print('Точность',estim, accuracy)

Точность 1 0.7402799377916018
Точность 4 0.7729393468118196
Точность 12 0.7869362363919129
Точность 20 0.7822706065318819
Точность 40 0.7931570762052877
Точность 80 0.7962674961119751
Точность 90 0.7916018662519441


In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=80)
model.fit(features_train, target_train) 
model_pred = model.predict(features_valid)
accuracy = accuracy_score(target_valid, model_pred)
print('Точность случайного леса валидационной выборки', accuracy)

Точность случайного леса валидационной выборки 0.7962674961119751


In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
model_pred = model.predict(features_valid)
accuracy = accuracy_score(target_valid, model_pred)
print('Точность логистической регрессии', accuracy)

Точность логистической регрессии 0.7402799377916018


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


 - В результате исследования выбираем модель с наибольшей точностью -  *модель дерева решений*

## 4. Проверьте модель на тестовой выборке

In [ ]:
model = DecisionTreeClassifier(random_state=12345, max_depth=7, min_samples_leaf=16)
model.fit(features_train, target_train)
model_pred = model.predict(features_test)
accuracy = accuracy_score(target_test, model_pred)
print('Точность дерева решений тестовой выборки:', accuracy)

Точность дерева решений тестовой выборки: 0.7838258164852255


- Проведено тестирование выбранной модели. Доля правильных ответов выше 0.75

## 5. (бонус) Проверьте модели на адекватность

In [ ]:
#проверим модель на вменяемость с помощью dummy classifier
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy='most_frequent', random_state=12345).fit(features_train, target_train)
dummy_pred = dummy.predict(features_test)
accuracy_dummy = accuracy_score(target_test, dummy_pred)
print(accuracy_dummy)

0.7060653188180405


 - После сравнения выбранной модели со случайной, видим что ее точность выше. Что показывает ценность проведенных исследований.